In [1]:
import pandas as pd
import time as t
from pathlib import Path
import xlwings as xw
import YM 
from shutil import copyfile

In [2]:
ts1 = t.time()

In [3]:
file_1c = t.strftime("%m%d") + "0900.xlsx"
file_path_1c = Path("D:/temp/__удалить/", file_1c)
file_path_cumulative = Path("D:/Analytik/1_Основное планирование/", "01_ИСХОДНИК-НОВ (выгрузка 1С).xlsx")
file_path_out_test = Path("D:/temp/", "01_ИСХОДНИК-НОВ (выгрузка 1С)_ТЕСТ.xlsx")
file_path_out_for_Teplitskaya = Path("D:\Общая\_ИСХ", "01_ИСХОДНИК-НОВ (выгрузка 1С).xlsx")

In [4]:
df_cf = pd.read_excel(file_path_cumulative, "TDSheet", usecols="A:O", index_col=False)
print("Записей в накопетельном файле: ", len(df_cf))

df_cf = df_cf.drop(df_cf.loc[df_cf['Категория клиента (св-во Контрагент)'] == "VIP Прямые продажи"].index)
df_cf = df_cf.drop(df_cf.loc[df_cf['Категория клиента (св-во Контрагент)'] == "Дистрибьютор"].index)
print("Последняя запись в накопителе от ", df_cf["По дням"].max())

df_1c = YM.read_excel_from_1c(file_path_1c, header=7, usecols="B:P", index_col=False)
print("Из 1с загружено строк: ", len(df_1c))

df_1с = df_1c.rename(columns={
                    'Контрагент (категории)':'Категория клиента (св-во Контрагент)'
                    ,'Пользователь':'Основной менеджер покупателя'
                    ,'Сумма продажи в ((2 грн))':'Сумма продажи в (грн)'
                    ,'Сумма продажи без скидок в ((2 грн))':'Сумма продажи без скидок в (грн)'
                    ,'Сумма скидки в ((2 грн))':'Сумма скидки в (грн)'
                     })
df_1с["По дням"] = pd.to_datetime(df_1с["По дням"], format="%d.%m.%Y")
df_vip1 = df_1с.loc[df_1с['Категория клиента (св-во Контрагент)'] == "VIP Прямые продажи"]
df_vip2 = df_1с.loc[df_1с['Категория клиента (св-во Контрагент)'] == "Дистрибьютор"]

df_vip = pd.concat([df_vip1, df_vip2])
df_1с = df_1с.drop(df_1с.loc[df_1с['Категория клиента (св-во Контрагент)'] == "VIP Прямые продажи"].index)
df_1c_retail = df_1с.drop(df_1с.loc[df_1с['Категория клиента (св-во Контрагент)'] == "Дистрибьютор"].index)
print("Последняя запись в базе 1с от ", df_1c_retail["По дням"].max())

df_cf_new = pd.concat([df_cf,
                       df_vip,
                       df_1c_retail.loc[df_1c_retail["По дням"] > df_cf["По дням"].max()]
                      ])
df_cf_new = df_cf_new.dropna(subset = ["Количество (в ед. отчетов)"])
df_cf_new.loc[df_cf_new["Контрагент"] == "Конечный оптовый покупатель", ["Грузополучатель"]] = ""
df_obj = df_cf_new.select_dtypes(['object'])
df_cf_new[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())

app_excel = xw.App(visible = True)
wb = xw.Book(file_path_cumulative)

ws = wb.sheets['TDSheet']
ws.range('A1').options(pd.DataFrame, index=False).value = df_cf_new
wb.api.RefreshAll()
wb.save()

copyfile(file_path_cumulative, file_path_out_for_Teplitskaya)

'''
wb.close()
app_excel.kill()
del app_excel
'''

print("Строк в новом файле: ", len(df_cf_new))
print("Время выполнения ", t.time() - ts1)

Записей в накопетельном файле:  238037
Последняя запись в накопителе от  2022-03-28 00:00:00
Файл перепакован из-за ошибки: "There is no item named 'xl/sharedStrings.xml' in the archive"
Из 1с загружено строк:  95710
Последняя запись в базе 1с от  2022-03-29 00:00:00
Строк в новом файле:  103891
Время выполнения  1320.3253939151764
